In [1]:
%run imports.py

Files already downloaded and verified
Preparing data..
Files already downloaded and verified
GPU available:  True
OS:  win32


In [4]:
#Set up Test Variables:

#Test batch size
num_batches=100

transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

testset = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True, transform=transform_test)

test_batch_size=int(numpy.shape(testset.data)[0]/num_batches)
#test_batch_size=100

testloader = torch.utils.data.DataLoader(
    testset, batch_size=test_batch_size, shuffle=False, num_workers=2)

test_features, test_labels = next(iter(testloader))
#Get num channels, width and height of input data
c,w,h=list(test_features[0].size())
#Batchsize, see load18.py or main.py for batch size
num_batches=int(numpy.shape(testset.data)[0]/test_batch_size)

from many_inj import progressive_inj_zero

Files already downloaded and verified


In [5]:
#For epoch 160, azure

#wds=[0,5e-7,5e-5,5e-3]
wds=[0]
sch="plateau"

tot=100000
step=10000
n_exp=5
bs=128

def inj_zero_plateau_epoch_160():

    for wd in wds:
        print("Inj iteration: " + str(wd))
        name = "bs_"+str(bs)+"_sch_"+sch + "_wd_"+str(wd)
        
        path="./checkpoints/"+ name
        path_best= path + "/"+name+"_epoch_160"

        model = model_actions.load(path_best)

        pfi_model = pfi_core(model.eval(), h, w, test_batch_size, c=c,debug=True,use_cuda=torch.cuda.is_available())

        res = many_inj.progressive_inj_zero(pfi_model,tot,step,n_exp=n_exp)

        save_name = name + "_inj_zero_tot_" + str(tot)+"_step_" + str(step) + "_nexp_"+ str(n_exp) + "_epoch_160"
        save_data={}
        save_data["avg"]=res[0]
        save_data["std"]=res[1]
        save_data["loss_avg"] = [2]
        save_data["loss_std"] = [3]
        utils.save(save_data,save_name)

In [6]:
az_manage_proc.run_and_delete(log.log_time,inj_zero_plateau_epoch_160)

Inj iteration: 0
Injection: 0 of 2
Injection: 1 of 2
Injection: 2 of 2
